In [52]:
import boto3

Setup Path File Parquet

In [ ]:
localFilePath = {
    'cuaca': '../dataStaging/dataCuaca.parquet',
    'pangan': '../dataStaging/dataPangan.parquet'
}

minioObjectName = {
    'cuaca': 'staging/dataCuaca.parquet',
    'pangan': 'staging/dataPangan.parquet'
}

Setup Kredensial MinIO

In [54]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'supply-chain-datalake',
    'endpoint': 'http://localhost:9000'
}

Inisialisasi Koneksi (Resource dan Client)

In [55]:
# Menggunakan Resource untuk fungsi level tinggi (seperti upload file)
s3Resource = boto3.resource(
    's3',
    endpoint_url = minioAuth['endpoint'],
    aws_access_key_id = minioAuth['accessKey'],
    aws_secret_access_key = minioAuth['secretKey'],
    verify = False # Wajib: Karena MinIO default di Docker itu http / tidak menggunakan SSL (https)
)

# Menggunakan Client untuk fungsi level rendah (seperti cek/buat bucket)
s3_client = boto3.client(
    's3',
    endpoint_url = minioAuth['endpoint'],
    aws_access_key_id = minioAuth['accessKey'],
    aws_secret_access_key= minioAuth['secretKey'],
    verify = False
)

Fungsi untuk memastikan bucket ada

In [56]:
def checkBucketIfExists(bucketName):
    try:
        
        # Jika sudah ada
        s3_client.head_bucket(Bucket = bucketName)
        print(f"Bucket '{bucketName}' sudah ada.")
        
    except Exception as e:
        
        # Jika bucket tidak ditemukan, buat bucket baru
        if '404' in str(e):
             s3_client.create_bucket(Bucket = bucketName)
             print(f"Bucket '{bucketName}' berhasil dibuat.")
             
        # Jika error bukan 404 (artinya ada kesalahan lain, biasanya salah endpoint api)
        else:
            print(f"Terjadi kesalahan saat memeriksa/membuat bucket: {e}")

checkBucketIfExists(minioAuth['bucket'])

Bucket 'supply-chain-datalake' berhasil dibuat.


In [57]:
# Upload file cuaca
s3Resource.Bucket(minioAuth['bucket']).upload_file(
    localFilePath['cuaca'], 
    minioObjectName['cuaca']
    )

# Upload file pangan
s3Resource.Bucket(minioAuth['bucket']).upload_file(
    localFilePath['pangan'], 
    minioObjectName['pangan']
    )

print(f"\n Berhasil mengunggah ke MinIO ✅")
print(f"\nAnda bisa melihat data pangan di GUI MinIO pada path: {minioAuth['bucket']}/{minioObjectName['pangan']}")
print(f"\nAnda bisa melihat data cuaca di GUI MinIO pada path: {minioAuth['bucket']}/{minioObjectName['cuaca']}")


 Berhasil mengunggah ke MinIO ✅

Anda bisa melihat data pangan di GUI MinIO pada path: supply-chain-datalake/staging/dataPangan.parquet

Anda bisa melihat data cuaca di GUI MinIO pada path: supply-chain-datalake/staging/dataCuaca.parquet
